<a href="https://colab.research.google.com/github/natrask/ENM5310/blob/main/CNN_VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I'm adopting here the [tensorflow tutorial for CNN VAEs](https://www.tensorflow.org/tutorials/generative/cvae) stripping out as much of the extra code as possible so we can focus just on how to construct the ELBO loss function and see the essential ingredients.

In [ ]:
import numpy as np
import scipy.sparse.linalg
import matplotlib.pyplot as plt


import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.reset_default_graph()
config = tf.ConfigProto()
sess = tf.Session(config=config)

Instructions for updating:
non-resource variables are not supported in the long term


Load the MNIST dataset consisting of a bunch of handwritten digits

In [ ]:
(train_images, _), (test_images, _) = tf.keras.datasets.mnist.load_data()
def preprocess_images(images):
  images = images.reshape((images.shape[0], 28, 28, 1)) / 255.
  return np.where(images > .5, 1.0, 0.0).astype('float32')

train_images = preprocess_images(train_images)
test_images = preprocess_images(test_images)

Break the dataset into a pile of test and training data

Build up the encoder architecture, using the reparameterization trick to generate a sample from the posterior distribution

In [ ]:
latent_dim = 2
npix1d = 28
Nbatch = 1
input_img = tf.placeholder(dtype=tf.float32,shape=(Nbatch,npix1d,npix1d))
encoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(28, 28, 1)),
            tf.keras.layers.Conv2D(
                filters=32, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.Conv2D(
                filters=64, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.Flatten(),
            # No activation
            tf.keras.layers.Dense(latent_dim + latent_dim),
        ])
decoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(latent_dim,)),
            tf.keras.layers.Dense(units=7*7*32, activation=tf.nn.relu),
            tf.keras.layers.Reshape(target_shape=(7, 7, 32)),
            tf.keras.layers.Conv2DTranspose(
                filters=64, kernel_size=3, strides=2, padding='same',
                activation='relu'),
            tf.keras.layers.Conv2DTranspose(
                filters=32, kernel_size=3, strides=2, padding='same',
                activation='relu'),
            # No activation
            tf.keras.layers.Conv2DTranspose(
                filters=1, kernel_size=3, strides=1, padding='same'),
        ])

#Now sample from the encoder to get Z, and decode to get Xhat
mu_q, logvar_q = tf.split(encoder(tf.expand_dims(input_img,-1)), num_or_size_splits=2, axis=1)
eps = tf.random.normal(shape=mu_q.shape)
Zsample = eps * tf.exp(logvar_q * .5) + mu_q
Xhat = decoder(Zsample)

Build ELBO loss function

In [ ]:
def log_normal_pdf(sample, mean, logvar, raxis=1):
  log2pi = tf.math.log(2. * np.pi)
  return tf.reduce_sum(
      -.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi),
      axis=raxis)
cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=Xhat, labels=tf.expand_dims(input_img,-1))
logpx_z = -tf.reduce_sum(cross_ent, axis=[1, 2, 3])
logpz = log_normal_pdf(Zsample, 0., 0.)
logqz_x = log_normal_pdf(Zsample, mu_q, logvar_q)

LOSS = -tf.reduce_mean(logpx_z + logpz - logqz_x)